## MLOps homework 1 - introduction

Download datasets

In [4]:
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  63.4M      0 --:--:-- --:--:-- --:--:-- 63.4M


In [5]:
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0  62.5M      0 --:--:-- --:--:-- --:--:-- 62.4M


In [2]:
import pandas as pd

Train data

In [45]:
df = pd.read_parquet('yellow_tripdata_2023-01.parquet')

In [47]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Populate duration, normalised to certain range

In [48]:
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).apply(lambda x: x.total_seconds() / 60.)

Standard deviation of duration:

In [49]:
df.duration.std()

42.59435124195458

Normalise duration to value range

In [52]:
df.duration = df.duration.apply(lambda x: x if 1.0 <= x <= 60.0 else None)

Value range - fraction of non-nulled

In [53]:
df.duration.count() * 100 / df.duration.shape[0]

98.1220282212598

In [54]:
df.duration.fillna(0, inplace=True)

Prepare train dataset

In [55]:
from sklearn.preprocessing import OneHotEncoder

In [56]:
X = df[['PULocationID', 'DOLocationID']]
y = df['duration']

In [57]:
ohe = OneHotEncoder(handle_unknown='infrequent_if_exist')
Xtrain = ohe.fit_transform(X)

In [58]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(Xtrain, y)

LinearRegression()

In [61]:
from math import sqrt
from sklearn.metrics import mean_squared_error
sqrt(mean_squared_error(reg.predict(Xtrain), y))

8.271405664649853

Play with test data

In [62]:
df2 = pd.read_parquet('yellow_tripdata_2023-02.parquet')

In [63]:
df2['duration'] = (df2.tpep_dropoff_datetime - df2.tpep_pickup_datetime).apply(lambda x: x.total_seconds() / 60.).apply(lambda x: x if 1.0 <= x <= 60.0 else 0)

In [64]:
Xt = df2[['PULocationID', 'DOLocationID']]
yt = df2['duration']

In [65]:
Xtest = ohe.transform(Xt)

In [66]:
math.sqrt(mean_squared_error(reg.predict(Xtest), yt))

8.485868247335624